In [1]:
import pandas as pd
import altair as alt

We're going to start by constructing a data frame with two streams of samples:  the "slow" samples grow by one for every timestamp step, and the "fast" samples grow by nineteen for every timestamp step.  Both streams of samples will grow more quickly (but at the same proportions) in last third of our of our dataset, with a large jump in both.

In [2]:
slow = list(range(1,100)) + [x * 2 for x in range(101,150)]
fast = [x * 19 for x in slow]
kinds = (["slow"] * len(slow)) + (["fast"] * len(fast))
indices = list(range(len(slow))) * 2
data = pd.DataFrame({"timestamp": indices,"sample": slow + fast, "kind": kinds})


In [3]:
data.sample(10)

,timestamp,sample,kind
191,43,836,fast
39,39,40,slow
295,147,5662,fast
20,20,21,slow
183,35,684,fast
187,39,760,fast
242,94,1805,fast
9,9,10,slow
154,6,133,fast
165,17,342,fast


If we plot our data on a linear scale, it isn't obvious that both streams of samples are always growing at the same rate (and, thus, that the proportion of fast to slow quantities never changes):

In [4]:
alt.Chart(data).mark_line().encode(alt.X("timestamp"), 
                                   alt.Y("sample"), 
                                   color="kind")


alt.Chart(...)

On a log scale, on the other hand, the slope between two points always corresponds to the rate of change between those points.  Put another way, doubling a quantity means moving a point by the same amount of vertical space no matter how large or small it is.  Since the slopes of these lines are the same, it is obvious that both are growing proportionally -- the vertical distance between both lines is constant.

In [5]:
alt.Chart(data).mark_line().encode(alt.X("timestamp"), 
                                   alt.Y("sample", scale=alt.Scale(type="log")), 
                                   color="kind").interactive()



alt.Chart(...)

Let's see what happens when the rates of change change over time.

In [6]:
import numpy as np
rates = [(1 + x) / (len(slow) / 6) for x in range(len(slow))]
slow2 = list(np.multiply(slow, rates))

In [7]:
temp = pd.DataFrame({"timestamp": list(range(len(slow))), 
                     "kind": ["less-slow"] * len(slow),
                     "sample": slow2})

data2 = pd.concat([temp, data[data["kind"] == "fast"]])

In [8]:
data2

,timestamp,kind,sample
0,0,less-slow,0.040541
1,1,less-slow,0.162162
2,2,less-slow,0.364865
3,3,less-slow,0.648649
4,4,less-slow,1.013514
...,...,...,...
291,143,fast,5510.000000
292,144,fast,5548.000000
293,145,fast,5586.000000
294,146,fast,5624.000000


When we plot the `fast` and `less-slow` series with a linear Y axis, it isn't particularly obvious that the proportion of each kind of sample changes over time even though the less-slow samples are growing five times as fast at the right side of the graph as they were on the left.

In [9]:
alt.Chart(data2).mark_line().encode(alt.X("timestamp"), 
                                    alt.Y("sample"), 
                                    color="kind")



alt.Chart(...)

If we use a log scale, on the other hand, we can see that the slopes diverge over time because the lines are closer to one another on the right side of the graph than they were on the left side of the graph.

In [10]:
alt.Chart(data2).mark_line().encode(alt.X("timestamp"), 
                                    alt.Y("sample", scale=alt.Scale(type="log")), 
                                    color="kind").interactive()




alt.Chart(...)